#1. Qualidade dos Dados

##Carga das tabelas

In [0]:
from pyspark.sql import functions as F

fact = spark.table("imdb_mvp.fact_title_rating")
dim_title = spark.table("imdb_mvp.dim_title")
dim_genre = spark.table("imdb_mvp.dim_genre")
dim_date = spark.table("imdb_mvp.dim_date")

print("Registros carregados:")
print("fact_title_rating:", fact.count())
print("dim_title:", dim_title.count())
print("dim_genre:", dim_genre.count())
print("dim_date:", dim_date.count())


Registros carregados:
fact_title_rating: 2341369
dim_title: 2341369
dim_genre: 3871466
dim_date: 153


##Schema e tipos de dados

In [0]:
print("Schema fact_title_rating")
fact.printSchema()

print("Schema dim_title")
dim_title.printSchema()

print("Schema dim_genre")
dim_genre.printSchema()

print("Schema dim_date")
dim_date.printSchema()


Schema fact_title_rating
root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: long (nullable = true)
 |-- year_key: integer (nullable = true)
 |-- titleType: string (nullable = true)

Schema dim_title
root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- is_adult: boolean (nullable = true)
 |-- runtimeMinutes: integer (nullable = true)
 |-- year: integer (nullable = true)

Schema dim_genre
root
 |-- tconst: string (nullable = true)
 |-- genre: string (nullable = true)

Schema dim_date
root
 |-- year: integer (nullable = true)
 |-- decade: integer (nullable = true)



##Análise de valores nulos

In [0]:
def missing_analysis(df, df_name):
    print(f"\nValores nulos em {df_name}:")
    total = df.count()
    for col in df.columns:
        nulls = df.filter(F.col(col).isNull()).count()
        pct = round((nulls / total) * 100, 2)
        print(f"{col}: {nulls} registros nulos ({pct}%)")

missing_analysis(fact, "fact_title_rating")
missing_analysis(dim_title, "dim_title")
missing_analysis(dim_genre, "dim_genre")
missing_analysis(dim_date, "dim_date")



Valores nulos em fact_title_rating:
tconst: 0 registros nulos (0.0%)
averageRating: 1643814 registros nulos (70.21%)
numVotes: 1643814 registros nulos (70.21%)
year_key: 184242 registros nulos (7.87%)
titleType: 0 registros nulos (0.0%)

Valores nulos em dim_title:
tconst: 0 registros nulos (0.0%)
titleType: 0 registros nulos (0.0%)
primaryTitle: 0 registros nulos (0.0%)
originalTitle: 0 registros nulos (0.0%)
is_adult: 0 registros nulos (0.0%)
runtimeMinutes: 937871 registros nulos (40.06%)
year: 184242 registros nulos (7.87%)

Valores nulos em dim_genre:
tconst: 0 registros nulos (0.0%)
genre: 0 registros nulos (0.0%)

Valores nulos em dim_date:
year: 0 registros nulos (0.0%)
decade: 0 registros nulos (0.0%)


##Análise de dados numéricos

In [0]:
fact.select(
    F.min("averageRating").alias("min_rating"),
    F.max("averageRating").alias("max_rating"),
    F.min("numVotes").alias("min_votes"),
    F.max("numVotes").alias("max_votes")
).show()

dim_title.select(
    F.min("runtimeMinutes").alias("min_runtime"),
    F.max("runtimeMinutes").alias("max_runtime"),
    F.min("year").alias("min_year"),
    F.max("year").alias("max_year")
).show()

dim_date.select(
    F.min("year").alias("min_year"),
    F.max("year").alias("max_year"),
    F.min("decade").alias("min_decade"),
    F.max("decade").alias("max_decade")
).show()


+----------+----------+---------+---------+
|min_rating|max_rating|min_votes|max_votes|
+----------+----------+---------+---------+
|       1.0|      10.0|        5|  3127637|
+----------+----------+---------+---------+

+-----------+-----------+--------+--------+
|min_runtime|max_runtime|min_year|max_year|
+-----------+-----------+--------+--------+
|          0|    3692080|    1874|    2115|
+-----------+-----------+--------+--------+

+--------+--------+----------+----------+
|min_year|max_year|min_decade|max_decade|
+--------+--------+----------+----------+
|    1874|    2115|      1870|      2110|
+--------+--------+----------+----------+



##Detecção de outliers

In [0]:
fact.select(
    F.expr("percentile(numVotes, 0.25)").alias("p25_votes"),
    F.expr("percentile(numVotes, 0.5)").alias("median_votes"),
    F.expr("percentile(numVotes, 0.75)").alias("p75_votes")
).show()

dim_title.select(
    F.expr("percentile(runtimeMinutes, 0.25)").alias("p25_runtime"),
    F.expr("percentile(runtimeMinutes, 0.5)").alias("median_runtime"),
    F.expr("percentile(runtimeMinutes, 0.75)").alias("p75_runtime")
).show()


+---------+------------+---------+
|p25_votes|median_votes|p75_votes|
+---------+------------+---------+
|     14.0|        34.0|    156.0|
+---------+------------+---------+

+-----------+--------------+-----------+
|p25_runtime|median_runtime|p75_runtime|
+-----------+--------------+-----------+
|       10.0|          29.0|       81.0|
+-----------+--------------+-----------+



##Análise de duplicidade de chaves

In [0]:
print("Duplicidade em fact_title_rating (tconst):",
      fact.groupBy("tconst").count().filter("count > 1").count())

print("Duplicidade em dim_title (tconst):",
      dim_title.groupBy("tconst").count().filter("count > 1").count())

print("Duplicidade em dim_genre (tconst, genre):",
      dim_genre.groupBy("tconst", "genre").count().filter("count > 1").count())

print("Duplicidade em dim_date (year):",
      dim_date.groupBy("year").count().filter("count > 1").count())


Duplicidade em fact_title_rating (tconst): 0
Duplicidade em dim_title (tconst): 0
Duplicidade em dim_genre (tconst, genre): 0
Duplicidade em dim_date (year): 0


##Integridade referencial

In [0]:
from pyspark.sql import functions as F

# Tabelas
fact = spark.table("imdb_mvp.fact_title_rating")
dim_title = spark.table("imdb_mvp.dim_title")
dim_genre = spark.table("imdb_mvp.dim_genre")
dim_date = spark.table("imdb_mvp.dim_date")

def check_fk(child_df, parent_df, fk_col, pk_col, rel_name):
    """
    child_df: tabela que contém a FK
    parent_df: tabela que contém a PK
    fk_col: nome da coluna FK em child_df
    pk_col: nome da coluna PK em parent_df
    rel_name: descrição da relação para impressão
    """
    print(f"===== RELAÇÃO {rel_name} =====")

    total_child = child_df.count()
    null_fk = child_df.filter(F.col(fk_col).isNull()).count()

    # órfãos considerando todos os registros
    orfa_all = child_df.join(
        parent_df,
        child_df[fk_col] == parent_df[pk_col],
        "left_anti"
    ).count()

    # órfãos considerando apenas registros com FK não nula
    child_not_null = child_df.filter(F.col(fk_col).isNotNull())
    orfa_not_null = child_not_null.join(
        parent_df,
        child_not_null[fk_col] == parent_df[pk_col],
        "left_anti"
    ).count()

    print(f"Total linhas na tabela com FK              : {total_child}")
    print(f"Linhas com FK nula                        : {null_fk}")
    print(f"Linhas sem correspondência (inclui nulos) : {orfa_all}")
    print(f"Linhas sem correspondência (FK não nula)  : {orfa_not_null}")
    print()

# fact.tconst -> dim_title.tconst
check_fk(
    child_df=fact,
    parent_df=dim_title,
    fk_col="tconst",
    pk_col="tconst",
    rel_name="fact.tconst -> dim_title.tconst"
)

# fact.year_key -> dim_date.year
check_fk(
    child_df=fact,
    parent_df=dim_date,
    fk_col="year_key",
    pk_col="year",
    rel_name="fact.year_key -> dim_date.year"
)

# dim_genre.tconst -> dim_title.tconst
check_fk(
    child_df=dim_genre,
    parent_df=dim_title,
    fk_col="tconst",
    pk_col="tconst",
    rel_name="dim_genre.tconst -> dim_title.tconst"
)


===== RELAÇÃO fact.tconst -> dim_title.tconst =====
Total linhas na tabela com FK              : 2341369
Linhas com FK nula                        : 0
Linhas sem correspondência (inclui nulos) : 0
Linhas sem correspondência (FK não nula)  : 0

===== RELAÇÃO fact.year_key -> dim_date.year =====
Total linhas na tabela com FK              : 2341369
Linhas com FK nula                        : 184242
Linhas sem correspondência (inclui nulos) : 184242
Linhas sem correspondência (FK não nula)  : 0

===== RELAÇÃO dim_genre.tconst -> dim_title.tconst =====
Total linhas na tabela com FK              : 3871466
Linhas com FK nula                        : 0
Linhas sem correspondência (inclui nulos) : 0
Linhas sem correspondência (FK não nula)  : 0



##Títulos sem gênero ou sem avaliação

In [0]:
# Títulos sem gênero
sem_genero = dim_title.join(dim_genre, "tconst", "left_anti").count()
print("Títulos sem gênero:", sem_genero)

# Títulos sem avaliação
sem_avaliacao = fact.filter(F.col("averageRating").isNull()).count()
print("Títulos sem avaliação:", sem_avaliacao)


Títulos sem gênero: 117263
Títulos sem avaliação: 1643814


##Datas superiores ao ano atual

In [0]:
from pyspark.sql import functions as F
from datetime import datetime

ano_atual = datetime.now().year

anos_futuros = (
    dim_date
    .filter(F.col("year") > ano_atual)
    .select("year")
    .distinct()
    .orderBy("year")
)

anos_futuros.show()

qtde_registros_futuros = (
    fact.join(dim_date, fact.year_key == dim_date.year, "inner")
        .filter(F.col("year") > ano_atual)
        .count()
)

print("Ano atual:", ano_atual)
print("Total de registros com ano futuro na fact:", qtde_registros_futuros)


+----+
|year|
+----+
|2026|
|2027|
|2028|
|2029|
|2030|
|2031|
|2032|
|2115|
+----+

Ano atual: 2025
Total de registros com ano futuro na fact: 3678


#2. Análises de Negócio

##1. Evolução do volume de lançamentos por ano

In [0]:
%sql
SELECT d.year,
       COUNT(DISTINCT f.tconst) AS qtd_titulos
FROM imdb_mvp.fact_title_rating f
JOIN imdb_mvp.dim_date d
  ON f.year_key = d.year
WHERE f.year_key IS NOT NULL
  AND d.year <= year(current_date())
  AND d.year >= 1940
GROUP BY d.year
ORDER BY d.year;


year,qtd_titulos
1940,2645
1941,2601
1942,2531
1943,2298
1944,2097
1945,2093
1946,2446
1947,2644
1948,2972
1949,3274


Databricks visualization. Run in Databricks to view.

##2. Títulos mais votados por década (a partir de 2000)

In [0]:
%sql
SELECT d.decade,
       t.primaryTitle,
       f.numVotes
FROM imdb_mvp.fact_title_rating f
JOIN imdb_mvp.dim_title t
  ON f.tconst = t.tconst
JOIN imdb_mvp.dim_date d
  ON f.year_key = d.year
WHERE f.year_key IS NOT NULL
  AND d.year <= year(current_date())
  AND d.year >= 2000
QUALIFY ROW_NUMBER() OVER (PARTITION BY d.decade ORDER BY f.numVotes DESC) <= 3
ORDER BY d.decade, f.numVotes DESC;


decade,primaryTitle,numVotes
2000,The Dark Knight,3103210
2000,Breaking Bad,2434902
2000,The Lord of the Rings: The Fellowship of the Ring,2160893
2010,Inception,2757756
2010,Game of Thrones,2506361
2010,Interstellar,2438660
2020,Dune: Part One,990589
2020,Spider-Man: No Way Home,988797
2020,Oppenheimer,960504


##3. Avaliação média por gênero

In [0]:
%sql
SELECT g.genre,
       ROUND(AVG(f.averageRating), 2) AS avg_rating,
       COUNT(*) AS qtd_titulos
FROM imdb_mvp.fact_title_rating f
JOIN imdb_mvp.dim_genre g
  ON f.tconst = g.tconst
WHERE f.averageRating IS NOT NULL
GROUP BY g.genre
ORDER BY avg_rating DESC;


genre,avg_rating,qtd_titulos
Documentary,7.08,107794
Biography,7.01,17397
History,6.99,18376
Music,6.84,17453
Short,6.8,158090
Sport,6.76,8952
Animation,6.64,43808
Family,6.59,33493
War,6.57,9735
Drama,6.53,264068


##4. Popularidade por gênero (volume total de votos)

In [0]:
%sql
SELECT g.genre,
       SUM(f.numVotes) AS total_votes
FROM imdb_mvp.fact_title_rating f
JOIN imdb_mvp.dim_genre g
  ON f.tconst = g.tconst
GROUP BY g.genre
ORDER BY total_votes DESC;


genre,total_votes
Drama,772952317
Action,448199458
Comedy,444177660
Adventure,369855310
Crime,296602540
Thriller,228819117
Romance,172184273
Sci-Fi,166125386
Mystery,161144239
Horror,144134013


##5. Relação entre duração e avaliação média

In [0]:
from pyspark.sql import functions as F

df_runtime_rating = (
    fact.join(dim_title, "tconst")
        .filter(F.col("runtimeMinutes").isNotNull())
        .filter(F.col("averageRating").isNotNull())
        .select("runtimeMinutes", "averageRating")
)

df_runtime_rating.describe(["runtimeMinutes", "averageRating"]).show()

corr = df_runtime_rating.stat.corr("runtimeMinutes", "averageRating")
print("Correlação entre duração e avaliação:", corr)

df_bins = (
    df_runtime_rating
      .withColumn(
          "runtime_bin",
          F.when(F.col("runtimeMinutes") < 60, "< 60")
           .when(F.col("runtimeMinutes").between(60, 89), "60–89")
           .when(F.col("runtimeMinutes").between(90, 119), "90–119")
           .when(F.col("runtimeMinutes").between(120, 149), "120–149")
           .when(F.col("runtimeMinutes").between(150, 179), "150–179")
           .otherwise(">= 180")
      )
      .groupBy("runtime_bin")
      .agg(
          F.count("*").alias("qtd_titulos"),
          F.round(F.avg("averageRating"), 2).alias("avg_rating")
      )
      .orderBy("runtime_bin")
)

df_bins.show()


+-------+-----------------+------------------+
|summary|   runtimeMinutes|     averageRating|
+-------+-----------------+------------------+
|  count|           571719|            571719|
|   mean|73.93119696914043| 6.458670080931281|
| stddev|4884.778395835496|1.3887439923232916|
|    min|                0|               1.0|
|    max|          3692080|              10.0|
+-------+-----------------+------------------+

Correlação entre duração e avaliação: 5.799109129078235e-05
+-----------+-----------+----------+
|runtime_bin|qtd_titulos|avg_rating|
+-----------+-----------+----------+
|    120–149|      30664|      6.54|
|    150–179|       8241|      6.71|
|      60–89|     140196|      6.15|
|     90–119|     154656|      6.12|
|       < 60|     231627|      6.84|
|     >= 180|       6335|       6.9|
+-----------+-----------+----------+



##6. Avaliação média por tipo de título

In [0]:
%sql
SELECT t.titleType,
       ROUND(AVG(f.averageRating), 2) AS avg_rating,
       COUNT(*) AS qtd_titulos
FROM imdb_mvp.fact_title_rating f
JOIN imdb_mvp.dim_title t
  ON f.tconst = t.tconst
WHERE f.averageRating IS NOT NULL
GROUP BY t.titleType
ORDER BY avg_rating DESC;


titleType,avg_rating,qtd_titulos
tvMiniSeries,6.99,23782
tvSeries,6.83,107485
short,6.81,174591
tvMovie,6.59,55768
movie,6.18,335929


##7. Evolução da avaliação média ao longo das décadas

In [0]:
%sql
SELECT d.decade,
       ROUND(AVG(f.averageRating), 2) AS avg_rating
FROM imdb_mvp.fact_title_rating f
JOIN imdb_mvp.dim_date d
  ON f.year_key = d.year
WHERE f.year_key IS NOT NULL
  AND d.year <= year(current_date())
  AND d.year >= 1940
  AND f.averageRating IS NOT NULL
GROUP BY d.decade
ORDER BY d.decade;


decade,avg_rating
1940,6.21
1950,6.29
1960,6.38
1970,6.21
1980,6.29
1990,6.33
2000,6.52
2010,6.66
2020,6.72


Databricks visualization. Run in Databricks to view.

##8. Crescimento dos gêneros ao longo do tempo

In [0]:
%sql
SELECT d.decade,
       g.genre,
       COUNT(*) AS qtd_titulos
FROM imdb_mvp.fact_title_rating f
JOIN imdb_mvp.dim_genre g
  ON f.tconst = g.tconst
JOIN imdb_mvp.dim_date d
  ON f.year_key = d.year
WHERE f.year_key IS NOT NULL
  AND d.year <= year(current_date())
  AND d.year >= 1940
GROUP BY d.decade, g.genre
ORDER BY d.decade, qtd_titulos DESC;


decade,genre,qtd_titulos
1940,Short,8573
1940,Drama,8133
1940,Comedy,5736
1940,Documentary,4706
1940,Animation,2126
1940,Romance,1981
1940,Music,1618
1940,Family,1443
1940,Crime,1355
1940,Musical,1233


Databricks visualization. Run in Databricks to view.